時間がかかるので引っ越し後にやってない。<br>
いけるはず…

In [1]:
import os
import pandas as pd
import numpy as np
import math
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import cm
from parse import *
import seaborn as sns

In [2]:
CHUNKSIZE = 100000

TRAIN_PATH        = "../../data/train_StationPathInfo.csv"
TRAIN_PATH_EX     = "../../data/train_StationPathInfoEx.csv"
TEST_PATH         = "../../data/test_StationPathInfo.csv"
TEST_PATH_EX      = "../../data/test_StationPathInfoEx.csv"

SEED = 0
CHUNKSIZE = 50000
NROWS = 1200000

ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
    
    
    

In [3]:
# 読み出す。

df_train_bin = pd.read_csv(TRAIN_PATH)
df_test_bin  = pd.read_csv(TEST_PATH)
df_pass = pd.concat([df_train_bin, df_test_bin])


通過ステーションをヒートマップにしてみる。

In [4]:
# 時系列でソートしたヒートマップを出してみる。

TEST_START_TIME      = "../../data/test_StartEndTime.csv"
TRAIN_START_TIME     = "../../data/train_StartEndTime.csv"

df_start_train = pd.read_csv(TRAIN_START_TIME)
df_start_train = df_start_train.ix[:,['Id','StartTime','EndTime', 'Response']]
df_start_test = pd.read_csv(TEST_START_TIME)
df_start_test['Response'] = -1


C:\Users\Tomonobu\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [5]:
df_start_train.head()

,Id,StartTime,EndTime,Response
0,4,82.24,87.29,0
1,6,1313.12,1315.75,0
2,7,1618.70,1624.42,0
3,9,1149.20,1154.16,0
4,11,602.64,606.02,0


In [6]:
df_start_test.head()
df_start = pd.concat([df_start_train, df_start_test])
df_start.head()

,Id,StartTime,EndTime,Response
0,4,82.24,87.29,0
1,6,1313.12,1315.75,0
2,7,1618.70,1624.42,0
3,9,1149.20,1154.16,0
4,11,602.64,606.02,0


In [7]:
df_train = pd.merge(df_start, df_pass, how='left')
df_train = df_train.fillna(-1500)




In [8]:
df_train[df_train.isnull().any(axis=1)]

,Id,StartTime,EndTime,Response,L0_S0_D1,L0_S1_D26,L0_S2_D34,L0_S3_D70,L0_S4_D106,L0_S5_D115,...,L3_S42_D4029,L3_S43_D4062,L3_S44_D4101,L3_S45_D4125,L3_S46_D4135,L3_S47_D4140,L3_S48_D4194,L3_S49_D4208,L3_S50_D4242,L3_S51_D4255


In [11]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, log_loss, make_scorer

def calc_mcc(cf_mat):
    tn, fp, fn, tp = cf_mat.ravel()
    print(tn, fp, fn, tp)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    return mcc

def mcc_scorer(y_true, y_pred):
    cf_mat = confusion_matrix(y_true, y_pred)
    return calc_mcc(cf_mat)

def separate_X_y(df):
    X = df.drop(['Response'], axis=1)
    y = df['Response']
    return X, y

def train_with_r_forest(df):
    X_train, y_train = separate_X_y(df)

    cl_weight = {0:0.3, 1:0.7}
    
    # Grid Search
#    params={'max_depth': [10, 25, 50],
#            'subsample': [0.8, 0.9, 0.95],
#            'colsample_bytree': [1.0],
#            'scale_pos_weight': [2, 3, 4, 5]
#    }
    params={'max_depth': [5, 10],
            'subsample': [0.9],
            'colsample_bytree': [1.0],
            'scale_pos_weight': [2]
    }

    xgb_model = xgb.XGBClassifier()
    gs = GridSearchCV(xgb_model,
                      params,
                      cv=5,
                      scoring={'mcc' : make_scorer(mcc_scorer)},
                      n_jobs=1,
                      verbose=2,
                      refit='mcc')

    gs.fit(X_train, y_train)
#rf = RandomForestClassifier(max_depth=10,n_estimators=50, random_state=33)
#    rf.fit(X_train, y_train)
    return gs


In [12]:



def train_rf_parameter(df):
    df = df[(df['Response'] == 0) | (df['Response'] == 1)]
    df_train, df_test = train_test_split(df, random_state=33)
    
    df_train_ok   = df_train[df_train['Response'] == 0]
    df_train_ng   = df_train[df_train['Response'] == 1]
    undersample_rate = len(df_train_ng) * 30 / len(df_train_ok)
    df_train_ok_sample = df_train_ok.sample(frac = undersample_rate)
    df_train = pd.concat([df_train_ok_sample, df_train_ng])

    rf = train_with_r_forest(df_train)
    X_test, y_test = separate_X_y(df_test)
    y_pred = rf.predict(X_test)
    print(y_pred)   
    
    cf_mat = confusion_matrix(y_test, y_pred)
    print(cf_mat)

    mcc = calc_mcc(cf_mat)
    print(mcc)    
    return rf

model = train_rf_parameter(df_train)




Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9 
31100 82 896 144
124410 318 3586 571
[CV]  colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9, total=  35.9s
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   37.2s remaining:    0.0s


31092 90 911 129
124429 299 3559 598
[CV]  colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9, total=  30.5s
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9 
31077 105 918 121
124423 305 3544 614
[CV]  colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9, total=  32.6s
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9 
31084 98 899 140
124419 309 3559 599
[CV]  colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9, total=  24.8s
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9 
31109 73 893 146
124392 336 3569 589
[CV]  colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9, total=  30.8s
[CV] colsample_bytree=1.0, max_depth=10, scale_pos_weight=2, subsample=0.9 
31121 61 886 154
124610 118 3268 889
[CV]  colsample_bytree=1.0, max_depth=10, scale_pos_weight=2, subsample=0.9, total= 1.1min
[CV] colsample_bytree=1.0, max_depth=10, scale_pos_weight=2, s

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  7.6min finished


[0 0 0 ..., 0 0 0]
[[293438    817]
 [  1425    257]]
293438 817 1425 257
0.187544961694


In [13]:
model.best_estimator_

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1.0,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=2, seed=0, silent=True, subsample=0.9)

In [14]:
def train_and_predict_one_chnuk(df_train):
    df_train_ok   = df_train[df_train['Response'] == 0]
    df_train_ng   = df_train[df_train['Response'] == 1]
    df_test = df_train[df_train['Response'] == -1]
    
    undersample_rate = len(df_train_ng) * 20 / len(df_train_ok)
    df_train_ok_sample = df_train_ok.sample(frac = undersample_rate)

    df_train_balance = pd.concat([df_train_ok_sample, df_train_ng])
    
    rf = train_with_r_forest(df_train_balance)

    X_test, y_test = separate_X_y(df_test)
    y_pred = rf.predict(X_test)
      
    df_result_add = pd.DataFrame(columns=['Id', 'Response'])
    df_result_add.loc[:, 'Id']       = df_test['Id'].values
    df_result_add.loc[:, 'Response'] = y_pred
    
    return df_result_add

df_result = train_and_predict_one_chnuk(df_train)
    

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9 
27453 63 1376 0
109714 350 4700 803
[CV]  colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9, total=  11.2s
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.6s remaining:    0.0s


27407 109 1376 0
109740 324 4668 835
[CV]  colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9, total=  11.0s
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9 
27415 101 1376 0
109738 326 4667 836
[CV]  colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9, total=  11.0s
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9 
27440 76 1376 0
109719 345 4635 868
[CV]  colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9, total=  11.0s
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9 
27430 86 1375 0
109719 345 4678 826
[CV]  colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.9, total=  11.2s
[CV] colsample_bytree=1.0, max_depth=10, scale_pos_weight=2, subsample=0.9 
27439 77 1376 0
109859 205 4257 1246
[CV]  colsample_bytree=1.0, max_depth=10, scale_pos_weight=2, subsample=0.9, total=  22.4s
[CV] colsample_bytree=1.0, max_depth=10, scale_pos_weight=2, subs

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  2.9min finished


In [15]:
df_result[df_result['Response']==1].shape


(5356, 2)

In [16]:
df_result[df_result['Response']==0].shape

(1178392, 2)

In [ ]:
df_result.to_csv("../../submission/submit_20180621_2.csv", index=False)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1.0,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=25,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=2, seed=0, silent=True, subsample=0.95)